In [106]:
import pandas as pd
import os
import nltk

import re
from readability import Readability

In [105]:
!pip install pyarrow
!pip install fastparquet
!pip install py-readability-metrics

In [28]:
#file_loc = "/Users/seanzhou/Documents/Intermediate Data/all_data.parquet"
df = pd.read_csv("all_data.csv")
df = df.loc[:, ["ticker", "earnings_call_date", "transcript"]]

# See an example
print(df['transcript'][0][:2000)

Operator: Good morning and thank you for standing by. Welcome to the AbbVie Second 2014 Earnings Conference Call. All participants will be able to listen-only. This conference is being recorded by AbbVie. I would now like to introduce Mr. Larry Peepo, Vice President of Investor Relations. You may begin sir.
Larry Peepo: Thank you. Good morning and thanks for joining us. On the call with me today are Rick Gonzalez, Chairman of the Board and Chief Executive Officer; and Bill Chase, Executive Vice President of Finance and Chief Financial Officer. Rick will begin by discussing AbbVie’s results from the second quarter and then provide an update on our pipeline and some of the key milestones we expect this year. Bill will give a more detailed review of our quarterly performance and then provide an overview of our 2014 outlook. As a reminder we are currently operating under the UK takeover code and will be until the Shire transaction is completed. The UK takeover code governs what we are able

# Feature 1: numeric transparency

Test Case:
In 2014, our company achieved significant milestones: we increased our staff to 150 employees, improved our efficiency by 13%, and reached a revenue of $75 million. During this period, our stock value appreciated by 15.23%, and we successfully launched a new product that generated $0.75 in earnings per share. Our research team, created 13 years ago, played a crucial role in this success, contributing insights that led to a market expansion of 255 new locations. Additionally, our long-term investments matured, yielding a substantial return of 234,455.23 dollars, setting us on a solid path for the next 18 months. $123,123.4. covid-19 and 10-K. 


In [98]:
def get_num_to_word(text):
    tokens = nltk.word_tokenize(text)
    types = set(tokens)

    # Get the number of word types (without numbers)
    # Matches integers (4 digits max) and floats without taking into account cases like covid-19 or 10-K
    re_pat = r'(?<![\w.-])\d{1,4}(?:,\d)*(?:\.\d+)?(?![\w.-])'

    # Delete the count of numeric vals in types to get only word types
    N_unique_number = sum(bool(re.match(re_pat, type)) for type in types)
    N_word_tokens = len(types) - N_unique_number    # Num of unique words
    N_num = sum(bool(re.match(re_pat, token)) for token in tokens)    # Count of all numeric vals

    #print(N_unique_number, N_num)
    numeric_transparency_ratio = N_num / N_word_tokens

    return numeric_transparency_ratio

In [99]:
sample = df['transcript'][100]
get_num_to_word(sample)

0.1672504378283713

In [100]:
df['num_transparency'] = [get_num_to_word(text) for text in df["transcript"]]

In [102]:
df.head()

,ticker,earnings_call_date,transcript,num_transparency
0,ABBV,2014-07-25,Operator: Good morning and thank you for stand...,0.104775
1,ABBV,2014-10-31,Operator: Good morning and thank you for stand...,0.073609
2,ABBV,2015-01-30,Operator: Good morning and thank you for stand...,0.127623
3,ABBV,2015-04-23,Operator: Good morning and thank you for stand...,0.084577
4,ABBV,2015-07-24,Operator: Good morning and thank you for stand...,0.098423


# Feature 2: Readability using Gunning-Fog Index

In [109]:
r = Readability(sample)

gf = r.gunning_fog()
print(gf.score)

11.896682367718638


In [110]:
def get_gf_score(text):
    r = Readability(text)
    gf = r.gunning_fog()
    
    return gf.score

df['readability'] = [get_gf_score(text) for text in df["transcript"]]

In [111]:
df.to_csv("all_data_NLP.csv")